# 05 - Combine Data and Prepare for Quantile Regression

This notebook combines and subsets the landslide triggering rainfall and annual maxima datasets.  After these steps, the landslide triggering rainfall data is ready for quantile regression.  


**Landslide triggering rainfall data**

This notebook combines the following datasets:  
1. lsdata_gsdr_rain.csv, which has rainfall data from GSDR gauges within 25 km of a landslide
2. lsdata_durban_rain.csv, which has rainfall data from the South African Weather Service within 25 km of a landslide.
3. lsdata_medellin_rain.csv, which has rainfall data for Medellin from IDEAM within 25 km of a landslide.

It then: 
1. Filters out gauges that have cumulative precipitation of <0.01 mm in the event period before the landslide
2. Finds the closest gauge with fewer than 10% nans in the event period.
3. Joins some other information about each landslide and each city
4. Subsets to only landslides with a spatial uncertainty of <25 km
5. Determines which cities have at least 5 landslides with rainfall data and that meet all criteria

**Annual maxima data**

This notebook combines the following datasets: 
1. annual_block_maxima.csv, with annual block maxima at a range of durations from GSDR stations
2. annual_block_maxima_durban.csv, with annual block maxima from stations in Durban
3. annual_block_maxima_medellin.csv, with annual block maxima from stations in Medellin

It then: 
1. Subsets to gauges that were associated with a landslide
2. Filters for annual maxima that have 90% complete data in the year recorded and where the maximum is >0.01 mm

**Extended Data Table 1**

This notebook also compiles information on the number of landslides, gauges, and lengths of records for Extended Data Table 1.

The outputs of this notebook are: 
- lsdata_rain.csv, which includes rainfall metrics for each landslide for quantile regression
- annual_block_maxima_ls_gauges.csv, which includes the annual block maxima at a range of durations for gauges associated with a landslide

*These outputs are read into 06_BayesianQuantileRegression*

- edtable1.csv, which contains the information for Extended Data Table 1.



In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
#set directory with datasets
datadir = ''

#set directory where results should be saved
resultsdir = ''

In [2]:
lsdata_rain = pd.read_csv('lsdata_rain.csv')

In [4]:
lsdata_rain.columns

Index(['Unnamed: 0', 'inventory', 'src_index', 'inventory_id',
       'inventory_id_name', 'lsidx', 'ID_HDC_G0', 'UC_NM_MN',
       'date_local_midnight_utc', 'Folder', 'OriginalID', 'NewID', 'Latitude',
       'Longitude', 'Recordlength(hours)', 'Recordlength(years)', 'StartDate',
       'EndDate', 'Missingdata(%)', 'geometry_y', 'flag', 'coverage',
       'station_dist', 'event_id', 'tr_start', 'tr_tpk', 'tr_ppk', 'tr_htopk',
       'tr_cptopk', 'tr_htoeod', 'tr_cptoeod', 'tr_nnantopk', 'tr_nnan_toeod',
       'e_start', 'e_htopk', 'e_cptopk', 'e_htoeod', 'e_cptoeod', 'e_nnantopk',
       'e_nnantoeod', 'tr_ante24h', 'tr_ante7d', 'tr_ante14d', 'tr_ante21d',
       'tr_ante28d', 'city', 'precip_source', 'perc_nan', 'UC_NM_LST', 'AREA',
       'GRGN_L1', 'GRGN_L2', 'CTR_MN_NM', 'E_KG_NM_LST', 'E_SL_LST', 'trigger',
       'type', 'material', 'spat_unc'],
      dtype='object')

### combine and subset landslide data

In [ ]:
#get all landslides with GSDR data
lsdata_gsdr_rain = pd.read_csv(datadir + 'lsdata_gsdr_rain.csv')

lsdata_gsdr_rain.drop(['Unnamed: 0'], inplace = True, axis = 1)

In [ ]:
#get durban landslides 
lsdata_durban_rain = pd.read_csv(datadir + 'lsdata_durban_rain.csv')

lsdata_durban_rain.drop(['Unnamed: 0'], inplace = True, axis = 1)

In [ ]:
#get medellin landslides
lsdata_medellin_rain = pd.read_csv(datadir + 'lsdata_medellin_rain.csv')

lsdata_medellin_rain.drop(['Unnamed: 0'], inplace = True, axis = 1)

In [ ]:
#concatenate all rainfall data to get a single dataframe
lsdata_all_rain = pd.concat([lsdata_gsdr_rain_updated, lsdata_durban_rain, lsdata_medellin_rain])

In [ ]:
#get landslide and gauge combinations with data and >0.01 mm of event rainfall
lsdata = lsdata_all_rain[(lsdata_all_rain['e_htopk'].notna()) & (lsdata_all_rain['e_cptopk']>0.01)].copy()

In [ ]:
#calculate the percentage of nans in the event period 
lsdata['perc_nan'] = lsdata['e_nnantopk']/lsdata['e_htopk']

In [ ]:
#get only events with fewer than 10% nans
lsdata = lsdata[lsdata['perc_nan'] < 0.1]

In [ ]:
#for each landslide, find the closest gauge.
targetdf = lsdata.sort_values('station_dist').drop_duplicates('lsidx')

### add attributes for the landslides from the global compilation and attributes for the cities from GHS-UCDB

In [ ]:
#Read global landslide compilation to extract relevant attributes for the considered landslides

landslides = pd.read_pickle(datadir + 'ls_urban_ts_rf_u.pkl')

In [ ]:
lsinfo = landslides.loc[:,['lsidx', 'trigger', 'type', 'material', 'spat_unc', 'geometry']]

In [ ]:
#remove duplicates

lsinfo = lsinfo.drop_duplicates()


In [ ]:
#join on some information for these cities

#get urban areas 
#read GHS geopackage (urban areas)
urban = gpd.read_file(datadir + '../GHS_STAT_UCDB2015MT_GLOBE_R2019A_V1_2/GHS_STAT_UCDB2015MT_GLOBE_R2019A/GHS_STAT_UCDB2015MT_GLOBE_R2019A_V1_2.gpkg', 
                       layer = 'GHS_STAT_UCDB2015MT_GLOBE_R2019A_V1_2')


#simplify 

urban = urban.loc[:, ['UC_NM_LST', 'ID_HDC_G0', 'AREA', 'GRGN_L1', 'GRGN_L2', 
                      'CTR_MN_NM', 'E_KG_NM_LST', 'E_SL_LST']]




In [ ]:
targetdf = targetdf.merge(urban, 
                         how = 'left', 
                         on = 'ID_HDC_G0')

In [ ]:
targetdf = targetdf.merge(lsinfo, 
                         how = 'left', 
                         on = 'lsidx')

get only landslides where the spatial uncertainty is <25 km 

In [ ]:
targetdf = targetdf[targetdf.apply(lambda row : row['spat_unc'] in ['<5km', '<1km', '<10km', '<25km', 'Exact'], axis = 1)]

### Select cities with at least 5 landslides that meet all criteria


- have a gauge within 25 km
- are rainfall triggered
- have available rainfall data
- have a spatial uncertainty of <25 km

In [ ]:
#counts of landslides in each city
count_per_city = targetdf.groupby('ID_HDC_G0').count()['inventory']

In [ ]:
#cities with >5 landslides
cities_gt5 = count_per_city[count_per_city >= 5]

In [ ]:
#subset to landslides in cities with >5 landslides
targetdf = targetdf[targetdf.apply(lambda row:row['ID_HDC_G0'] in cities_gt5, axis = 1)]

In [ ]:
#save to csv
targetdf.to_csv(resultsdir + 'lsdata_rain.csv')

### Get annual block maxima for gauges associated with landslides

In [ ]:
ann_block_max_gsdr = pd.read_csv(datadir + 'annual_block_maxima.csv')

In [ ]:
ann_block_max_gsdr['Date'] = ann_block_max_gsdr['Unnamed: 0']

In [ ]:
ann_block_max_gsdr.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [ ]:
ann_block_max_durban = pd.read_csv(resultsdir + 'annual_block_maxima_durban.csv')

In [ ]:
ann_block_max_medellin = pd.read_csv(resultsdir + 'annual_block_maxima_medellin.csv')

In [ ]:
ann_block_max = pd.concat([ann_block_max_gsdr, ann_block_max_durban, ann_block_max_medellin])

In [ ]:
#strip city names of white space, special characters, etc for R

ann_block_max['city'] = ann_block_max.apply(lambda row:''.join(e for e in row['UC_NM_MN'] if e.isalnum()), 
                                       axis = 1)


In [ ]:
ann_block_max['OriginalID'] = ann_block_max['OriginalID'].astype(str)

In [ ]:
#get only the gauges that we use in the analysis
ls_gauges_unique = pd.DataFrame(targetdf['OriginalID'].unique(), columns = ['OriginalID'])

In [ ]:
#subset to those gauges
ann_block_max_ls_gauges = pd.merge(ls_gauges_unique, 
                                  ann_block_max, 
                                  how = 'left', 
                                  on = 'OriginalID')

In [ ]:
#get only entries that have a 90% complete data in each year and where the block max is >0.01
ann_block_max_ls_gauges_data = ann_block_max_ls_gauges[(ann_block_max_ls_gauges['raw_notnainmw'] >= 8760*0.90) & 
                                                      (ann_block_max_ls_gauges['raw_block_max'] >= 0.01)]

In [ ]:
#remove medellin gauge 27015310 for year 2021 - there is a problem with this data
#no landslides were associated with that gauge in 2021, so this only affects the block maxima, not 
#the landslide data
ann_block_max_ls_gauges_data = ann_block_max_ls_gauges_data[(ann_block_max_ls_gauges_data['OriginalID'] != '27015310')
                                                           & (ann_block_max_ls_gauges_data['year'] != 2021)]

In [ ]:
ann_block_max_ls_gauges_data.to_csv(resultsdir + 'annual_block_maxima_ls_gauges.csv')

### Process data for Extended Data Table 1

In [ ]:
#Number of landslides in each city, earliest and latest landslide on record for each city

targetdf['date_local_midnight_utc'] = pd.to_datetime(targetdf['date_local_midnight_utc'])

targetdf['year'] = targetdf['date_local_midnight_utc'].dt.year

nls = targetdf.groupby('city').count()['lsidx']

earlyls = targetdf[['city', 'year']].groupby('city').min()['year']

latels = targetdf[['city', 'year']].groupby('city').max()['year']

In [ ]:
#Number of gauges in each city, earliest and latest year of precipitation records

earlygauge = ann_block_max_ls_gauges_data[['city', 'year']].groupby('city').min()['year']

lategauge = ann_block_max_ls_gauges_data[['city', 'year']].groupby('city').max()['year']

ann_block_max_ls_gauges_data_unique = ann_block_max_ls_gauges_data.drop_duplicates(subset = ['OriginalID', 'city'])

ngauge = ann_block_max_ls_gauges_data_unique.groupby('city').count()['OriginalID']

In [ ]:
#Extended data table 1
edtable1 = pd.DataFrame([nls, earlyls, latels, ngauge, earlygauge, lategauge]).transpose()

In [ ]:
edtable1.to_csv('edtable1.csv')